In [1]:
import pickle
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [3]:
df = pd.read_csv('../data/tcga_processed.tsv.gz', sep='\t')

### some variants are repeated multiple times
df = df.drop(columns=['Cancer_type', 'Cancer_type_count', 'NMF_cluster'])

agg_columns = [
    'NMD_efficiency', 't_vaf', 't_depth', 'n_vaf', 'n_depth',
    'VAF_RNA', 'depth_RNA', 'VAF_DNA_RNA_ratio', 'tpm_unstranded_x'
]

# Group by all other columns except the ones to aggregate
group_by_columns = [col for col in df.columns if col not in agg_columns]

# Perform grouping and aggregation without dropping NaNs
df_unq = df.groupby(
    group_by_columns, dropna=False, as_index=False
).agg({col: 'mean' for col in agg_columns})

df_unq

,build,chromosome,start,end,Hugo_Symbol,Transcript_ID,HGVSc,HGVSp,PTC_pos_codon,PTC_to_start_codon,...,var_token_idx,NMD_efficiency,t_vaf,t_depth,n_vaf,n_depth,VAF_RNA,depth_RNA,VAF_DNA_RNA_ratio,tpm_unstranded_x
0,GRCh38,chr1,944753,944753,NOC2L,ENST00000327044,c.2191C>T,p.Gln731Ter,731,2193,...,2240,-0.508648,0.311111,45.0,0.0,43.0,0.442623,244.0,1.422717,76.8018
1,GRCh38,chr1,952113,952113,NOC2L,ENST00000327044,c.1218G>A,p.Trp406Ter,406,1218,...,1267,1.629509,0.390244,41.0,0.0,14.0,0.126126,111.0,0.323198,49.1731
2,GRCh38,chr1,1255304,1255304,UBE2J2,ENST00000349431,c.679G>T,p.Gly227Ter,227,681,...,898,0.047557,0.480000,50.0,0.0,32.0,0.464435,239.0,0.967573,47.4110
3,GRCh38,chr1,1338573,1338573,DVL1,ENST00000378888,c.1288G>T,p.Glu430Ter,430,1290,...,1572,1.505167,0.413043,46.0,0.0,63.0,0.145511,323.0,0.352289,50.0535
4,GRCh38,chr1,1387314,1387314,CCNL2,ENST00000400809,c.1480C>T,p.Arg494Ter,494,1482,...,1485,-0.424007,0.407692,130.0,0.0,152.0,0.546980,298.0,1.341649,27.1883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4088,GRCh38,chrX,152920717,152920717,ZNF185,ENST00000370268,c.622C>T,p.Gln208Ter,208,624,...,658,1.785660,0.328358,67.0,0.0,65.0,0.095238,42.0,0.290043,52.3474
4089,GRCh38,chrX,153650171,153650171,DUSP9,ENST00000342782,c.1021G>T,p.Glu341Ter,341,1023,...,1285,-1.360998,0.389313,131.0,0.0,185.0,1.000000,35.0,2.568627,16.9024
4090,GRCh38,chrX,154030948,154030948,MECP2,ENST00000303391,c.880C>T,p.Arg294Ter,294,882,...,1129,-1.166585,0.406977,86.0,0.0,70.0,0.913580,81.0,2.244797,11.5544
4091,GRCh38,chrX,154354015,154354015,FLNA,ENST00000369850,c.5586C>A,p.Tyr1862Ter,1862,5586,...,5834,0.332048,0.904762,189.0,0.0,134.0,0.718750,32.0,0.794408,63.2541


### Splits

In [4]:
test_var_ids = df_unq[df_unq['chromosome'].isin(['chr20', 'chr21', 'chr22'])]['var_id'].values.tolist()
test_var_ids = list(set(test_var_ids))
test_var_ids[0:5], len(test_var_ids)

(['ENST00000263207:c.433G>T',
  'ENST00000359003:c.6013G>T',
  'ENST00000279068:c.682C>T',
  'ENST00000217244:c.916C>T',
  'ENST00000373271:c.3826C>T'],
 232)

In [5]:
val_var_ids = df_unq[df_unq['chromosome']=='chr19']['var_id'].values.tolist()
val_var_ids = list(set(val_var_ids))
val_var_ids[0:5], len(val_var_ids)

(['ENST00000245222:c.1709G>A',
  'ENST00000541714:c.2695C>T',
  'ENST00000356487:c.1576G>T',
  'ENST00000221735:c.1504C>T',
  'ENST00000222145:c.2839C>T'],
 210)

In [6]:
train_var_ids = list(set(df_unq.var_id.values.tolist()) - set(val_var_ids) - set(test_var_ids))
train_var_ids[0:5], len(train_var_ids)

(['ENST00000380006:c.10C>T',
  'ENST00000304698:c.2449C>T',
  'ENST00000394434:c.2326G>T',
  'ENST00000410053:c.1345G>T',
  'ENST00000356657:c.456G>A'],
 3651)

In [7]:
# create dictionaries with NMD_efficiency as values
test_dict = {}
val_dict = {}
train_dict = {}
var_pos_idx_dict = {}

for index, row in df_unq.iterrows():
    if row['var_id'] in test_var_ids:
        test_dict[row['var_id']] = row['NMD_efficiency']
    elif row['var_id'] in val_var_ids:
        val_dict[row['var_id']] = row['NMD_efficiency']
    else:
        train_dict[row['var_id']] = row['NMD_efficiency']

    var_pos_idx_dict[row['var_id']] = row['var_token_idx']

### Read embeddings

In [8]:
with open('../data/tcga_ref_embeds.pkl', 'rb') as file:
    ref_embeds = pickle.load(file)

with open('../data/tcga_alt_embeds.pkl', 'rb') as file:
    alt_embeds = pickle.load(file)

# create alt - ref embeds
altref_embeds = {}

for var_id, alt_embed in alt_embeds.items():
    altref_embeds[var_id] = alt_embed - ref_embeds[var_id]

### Train and evaluate

In [9]:
import logging
import sys
from collections import OrderedDict
from typing import Any, Dict, List, Optional

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr
import torch.optim as optim
import pandas as pd

# ---------------------------
# Hyperparameters
# ---------------------------
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 5e-4
HIDDEN_DIMS_phi = [8, 8]
DROPOUT = 0.25
N_EPOCHS = 50
EARLY_STOPPING_PATIENCE = 10
VALIDATION_INTERVAL = 5  # Perform validation every 5 epochs

# ---------------------------
# Dataset Definition
# ---------------------------
class DeepSetDataset(Dataset):
    def __init__(self, embeds_dict, target_dict, var_pos_idx_dict=None):
        self.ids = list(target_dict.keys())
        self.embeds = [embeds_dict[i] for i in self.ids]
        self.targets = [target_dict[i] for i in self.ids]
        self.var_pos_idx = [var_pos_idx_dict[i] if var_pos_idx_dict else None for i in self.ids]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        embed = self.embeds[idx]
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        var_pos_idx = self.var_pos_idx[idx]
        return embed, target, var_pos_idx


def collate_fn(batch):
    embeds, targets, var_pos_idx = zip(*batch)
    embeds = [torch.as_tensor(embed, dtype=torch.float32) for embed in embeds]
    embeds_padded = torch.nn.utils.rnn.pad_sequence(embeds, batch_first=True, padding_value=0.0)
    targets = torch.tensor(targets, dtype=torch.float32)
    mask = (embeds_padded != 0).any(dim=-1).float()
    return embeds_padded, targets, mask, var_pos_idx

# ---------------------------
# DeepSet Model without Rho
# ---------------------------
class DeepSetModel_noRho(nn.Module):
    def __init__(
        self,
        input_dim: int,
        phi_hidden_dims: List[int],
        output_dim: int = 1,
        activation: str = "ReLU",
        pool: str = "mean",
        dropout: Optional[float] = None,
    ):
        super().__init__()
        self.activation = getattr(nn, activation)()
        self.pool = pool
        self.dropout = nn.Dropout(dropout) if dropout else None
        
        # Phi network
        phi = []
        in_dim = input_dim
        for i, dim in enumerate(phi_hidden_dims):
            phi.append((f"phi_linear_{i}", nn.Linear(in_dim, dim)))
            if dropout:
                phi.append((f"phi_dropout_{i}", self.dropout))
            phi.append((f"phi_activation_{i}", self.activation))
            in_dim = dim
        phi.append((f"phi_output", nn.Linear(in_dim, output_dim)))
        self.phi = nn.Sequential(OrderedDict(phi))

    def forward(self, x, mask, var_pos_idx=None):
        x = self.phi(x)
        
        if self.pool == "sum":
            x = (x * mask.unsqueeze(-1)).sum(dim=1)
        elif self.pool == "max":
            x = (x * mask.unsqueeze(-1)).masked_fill(mask.unsqueeze(-1) == 0, float('-inf')).max(dim=1).values
        elif self.pool == "mean":
            x = (x * mask.unsqueeze(-1)).sum(dim=1) / (mask.sum(dim=1, keepdim=True) + 1e-8)
        elif self.pool == "token" and var_pos_idx is not None:
            x = torch.stack([x[i, idx] for i, idx in enumerate(var_pos_idx)])
        else:
            raise ValueError(f"Unsupported pooling operation: {self.pool}")
        
        return x



# ---------------------------
# Evaluation Function
# ---------------------------
'''def evaluate_regression_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    spearman_corr, _ = spearmanr(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return {
        'spearman_corr': spearman_corr,
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    }'''

# Evaluation Metrics
def evaluate_regression_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy().flatten()
    y_pred = y_pred.cpu().numpy().flatten()
    loss = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(loss)
    r2 = r2_score(y_true, y_pred)
    
    try:
        spearman_corr, _ = spearmanr(y_true, y_pred)
    except ValueError:
        spearman_corr = np.nan

    try:
        pearson_corr, _ = pearsonr(y_true, y_pred)
    except ValueError:
        pearson_corr = np.nan
    
    return {
        'loss': loss,
        'spearman_corr': spearman_corr,
        'pearson_corr': pearson_corr,
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    }


# ---------------------------
# Training Function
# ---------------------------
def train_model(train_dict, val_dict, test_dict, embeds_dict, var_pos_idx_dict=None, pool='mean'):
    train_loader = DataLoader(DeepSetDataset(embeds_dict, train_dict, var_pos_idx_dict), batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(DeepSetDataset(embeds_dict, val_dict, var_pos_idx_dict), batch_size=32, collate_fn=collate_fn)
    test_loader = DataLoader(DeepSetDataset(embeds_dict, test_dict, var_pos_idx_dict), batch_size=32, collate_fn=collate_fn)
    
    model = DeepSetModel_noRho(
        input_dim=embeds_dict[list(embeds_dict.keys())[0]].shape[1],
        phi_hidden_dims=HIDDEN_DIMS_phi,
        pool=pool,
        dropout=DROPOUT
    )
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.MSELoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    metrics = {'epoch': [], 'phase': [], 'loss': [], 'spearman_corr': [], 'pearson_corr':[], 'mae': [], 'rmse': [], 'r2': []}
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(1, N_EPOCHS + 1):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = val_loader
            
            all_preds, all_targets = [], []
            running_loss = 0.0
            
            for embeds, targets, mask, var_pos_idx in loader:
                embeds, targets, mask = embeds.to(device), targets.to(device), mask.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(embeds, mask, var_pos_idx).squeeze()
                    loss = criterion(outputs, targets)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * embeds.size(0)
                all_preds.append(outputs.detach())
                all_targets.append(targets.detach())
            
            epoch_loss = running_loss / len(loader.dataset)
            all_preds = torch.cat(all_preds)
            all_targets = torch.cat(all_targets)
            
            epoch_metrics = evaluate_regression_metrics(all_targets, all_preds)
            epoch_metrics['loss'] = epoch_loss
            
            # Early stopping for validation phase
            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= EARLY_STOPPING_PATIENCE:
                        print(f"Early stopping at epoch {epoch}")
                        metrics_df = pd.DataFrame(metrics)
                        test_metrics = evaluate_regression_metrics(all_targets, all_preds)
                        test_metrics['loss'] = epoch_loss
                        return metrics_df, test_metrics
            
            # Log metrics every 5 epochs
            if epoch % 5 == 0:
                print(
                    f"[Epoch {epoch:03d}] Phase: {phase:5s} | "
                    f"Loss: {epoch_metrics['loss']:.4f} | "
                    f"Spearman: {epoch_metrics['spearman_corr']:.4f} | "
                    f"Pearson: {epoch_metrics['pearson_corr']:.4f} | "
                    f"MAE: {epoch_metrics['mae']:.4f} | "
                    f"RMSE: {epoch_metrics['rmse']:.4f} | "
                    f"R²: {epoch_metrics['r2']:.4f}"
                )
                metrics['epoch'].append(epoch)
                metrics['phase'].append(phase)
                for key, value in epoch_metrics.items():
                    metrics[key].append(value)
    
    # Final Test Evaluation
    model.eval()
    all_preds, all_targets = [], []
    running_loss = 0.0
    
    for embeds, targets, mask, var_pos_idx in test_loader:
        embeds, targets, mask = embeds.to(device), targets.to(device), mask.to(device)
        outputs = model(embeds, mask, var_pos_idx).squeeze()
        loss = criterion(outputs, targets)
        
        running_loss += loss.item() * embeds.size(0)
        all_preds.append(outputs.detach())
        all_targets.append(targets.detach())
    
    epoch_loss = running_loss / len(test_loader.dataset)
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    
    test_metrics = evaluate_regression_metrics(all_targets, all_preds)
    test_metrics['loss'] = epoch_loss
    
    metrics_df = pd.DataFrame(metrics)
    return metrics_df, test_metrics


In [10]:
### Ref embeds

print("START TRAINING REF MEAN")
ref_mean_metrics_df, ref_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='mean'  
    )
ref_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/ref_mean_history_df.csv')

print("START TRAINING REF MAX")
ref_max_metrics_df, ref_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='max'  
    )
ref_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/ref_max_metrics_df.csv')

print("START TRAINING REF TOKEN")
ref_token_metrics_df, ref_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
ref_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/ref_token_metrics_df.csv')

print("START TRAINING REF SUM")
ref_sum_metrics_df, ref_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='sum'  
    )
ref_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/ref_sum_metrics_df.csv')

START TRAINING REF MEAN
[Epoch 005] Phase: train | Loss: 1.5586 | Spearman: 0.1374 | Pearson: 0.1389 | MAE: 1.0426 | RMSE: 1.2484 | R²: 0.0152
[Epoch 005] Phase: val   | Loss: 1.6970 | Spearman: 0.3720 | Pearson: 0.3631 | MAE: 1.0454 | RMSE: 1.3027 | R²: 0.0328
[Epoch 010] Phase: train | Loss: 1.4432 | Spearman: 0.3404 | Pearson: 0.3476 | MAE: 1.0007 | RMSE: 1.2013 | R²: 0.0881
[Epoch 010] Phase: val   | Loss: 1.5489 | Spearman: 0.4375 | Pearson: 0.4374 | MAE: 0.9760 | RMSE: 1.2445 | R²: 0.1172
[Epoch 015] Phase: train | Loss: 1.3682 | Spearman: 0.3719 | Pearson: 0.3767 | MAE: 0.9642 | RMSE: 1.1697 | R²: 0.1355
[Epoch 015] Phase: val   | Loss: 1.4223 | Spearman: 0.4614 | Pearson: 0.4562 | MAE: 0.9526 | RMSE: 1.1926 | R²: 0.1893
[Epoch 020] Phase: train | Loss: 1.3341 | Spearman: 0.3960 | Pearson: 0.3983 | MAE: 0.9446 | RMSE: 1.1550 | R²: 0.1570
[Epoch 020] Phase: val   | Loss: 1.3835 | Spearman: 0.4729 | Pearson: 0.4681 | MAE: 0.9171 | RMSE: 1.1762 | R²: 0.2114
[Epoch 025] Phase: train

In [11]:
### Alt embeds

print("START TRAINING ALT MEAN")
alt_mean_metrics_df, alt_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='mean'  
    )
alt_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/alt_mean_metrics_df.csv')

print("START TRAINING ALT MAX")
alt_max_metrics_df, alt_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='max'  
    )
alt_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/alt_max_metrics_df.csv')

print("START TRAINING ALT TOKEN")
alt_token_metrics_df, alt_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
alt_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/alt_token_metrics_df.csv')

print("START TRAINING ALT SUM")
alt_sum_metrics_df, alt_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='sum'  
    )
alt_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/alt_sum_metrics_df.csv')

START TRAINING ALT MEAN
[Epoch 005] Phase: train | Loss: 1.4438 | Spearman: 0.3593 | Pearson: 0.3581 | MAE: 1.0007 | RMSE: 1.2016 | R²: 0.0877
[Epoch 005] Phase: val   | Loss: 1.5431 | Spearman: 0.4301 | Pearson: 0.4408 | MAE: 1.0151 | RMSE: 1.2422 | R²: 0.1204
[Epoch 010] Phase: train | Loss: 1.3429 | Spearman: 0.3973 | Pearson: 0.3958 | MAE: 0.9519 | RMSE: 1.1589 | R²: 0.1514
[Epoch 010] Phase: val   | Loss: 1.4103 | Spearman: 0.4569 | Pearson: 0.4624 | MAE: 0.9469 | RMSE: 1.1875 | R²: 0.1962
[Epoch 015] Phase: train | Loss: 1.3190 | Spearman: 0.4111 | Pearson: 0.4097 | MAE: 0.9364 | RMSE: 1.1485 | R²: 0.1666
[Epoch 015] Phase: val   | Loss: 1.3823 | Spearman: 0.4731 | Pearson: 0.4750 | MAE: 0.9190 | RMSE: 1.1757 | R²: 0.2121
[Epoch 020] Phase: train | Loss: 1.3026 | Spearman: 0.4203 | Pearson: 0.4218 | MAE: 0.9283 | RMSE: 1.1413 | R²: 0.1769
[Epoch 020] Phase: val   | Loss: 1.3429 | Spearman: 0.4902 | Pearson: 0.4889 | MAE: 0.9162 | RMSE: 1.1588 | R²: 0.2346
[Epoch 025] Phase: train

In [12]:
### Alt - Ref embeds

print("START TRAINING ALTREF MEAN")
altref_mean_metrics_df, altref_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='mean'  
    )
altref_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/altref_mean_metrics_df.csv')

print("START TRAINING ALTREF MAX")
altref_max_metrics_df, altref_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='max'  
    )
altref_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/altref_max_metrics_df.csv')

print("START TRAINING ALTREF TOKEN")
altref_token_metrics_df, altref_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
altref_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/altref_token_metrics_df.csv')

print("START TRAINING ALTREF SUM")
altref_sum_metrics_df, altref_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='sum'  
    )
altref_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset_norho/altref_sum_metrics_df.csv')

START TRAINING ALTREF MEAN
[Epoch 005] Phase: train | Loss: 1.5692 | Spearman: 0.2031 | Pearson: 0.1678 | MAE: 1.0388 | RMSE: 1.2527 | R²: 0.0085
[Epoch 005] Phase: val   | Loss: 1.7162 | Spearman: 0.3852 | Pearson: 0.3337 | MAE: 1.0662 | RMSE: 1.3100 | R²: 0.0218
[Epoch 010] Phase: train | Loss: 1.4367 | Spearman: 0.4062 | Pearson: 0.3503 | MAE: 0.9931 | RMSE: 1.1986 | R²: 0.0922
[Epoch 010] Phase: val   | Loss: 1.6241 | Spearman: 0.3492 | Pearson: 0.2870 | MAE: 1.0233 | RMSE: 1.2744 | R²: 0.0743
[Epoch 015] Phase: train | Loss: 1.3360 | Spearman: 0.4593 | Pearson: 0.4127 | MAE: 0.9453 | RMSE: 1.1559 | R²: 0.1558
[Epoch 015] Phase: val   | Loss: 1.5403 | Spearman: 0.4029 | Pearson: 0.3527 | MAE: 0.9800 | RMSE: 1.2411 | R²: 0.1220
[Epoch 020] Phase: train | Loss: 1.2870 | Spearman: 0.4835 | Pearson: 0.4402 | MAE: 0.9210 | RMSE: 1.1344 | R²: 0.1868
[Epoch 020] Phase: val   | Loss: 1.5035 | Spearman: 0.4364 | Pearson: 0.3836 | MAE: 0.9721 | RMSE: 1.2262 | R²: 0.1430
[Epoch 025] Phase: tr

In [13]:
mean_metrics = pd.DataFrame({ 'ref':ref_mean_test_metrics,
                'alt': alt_mean_test_metrics, 
                'altref': altref_mean_test_metrics})

mean_metrics.to_csv('../res/metrics/test_deepset_norho/mean_metrics.csv')
mean_metrics

,ref,alt,altref
loss,1.389480,1.358351,1.399738
spearman_corr,0.472282,0.466998,0.462107
pearson_corr,0.443535,0.436449,0.385084
mae,0.890945,0.890734,0.930894
rmse,1.178762,1.165483,1.183105
r2,0.145854,0.164989,0.139548


In [14]:
max_metrics = pd.DataFrame({ 'ref':ref_max_test_metrics,
                'alt': alt_max_test_metrics, 
                'altref': altref_max_test_metrics})

max_metrics.to_csv('../res/metrics/test_deepset_norho/max_metrics.csv')
max_metrics

,ref,alt,altref
loss,1.622690,1.642522,1.724663
spearman_corr,0.463316,0.457282,0.187671
pearson_corr,0.444923,0.450530,0.209485
mae,0.942375,0.944849,1.039607
rmse,1.273849,1.281609,1.313264
r2,0.075095,0.063791,0.016972


In [15]:
token_metrics = pd.DataFrame({'ref':ref_token_test_metrics,
                'alt': alt_token_test_metrics, 
                'altref': altref_token_test_metrics})

token_metrics.to_csv('../res/metrics/test_deepset_norho/token_metrics.csv')
token_metrics

,ref,alt,altref
loss,1.680131,1.654160,1.681668
spearman_corr,0.231788,0.307548,0.242521
pearson_corr,0.207613,0.244855,0.212494
mae,1.043321,1.038370,1.032396
rmse,1.296199,1.286142,1.296792
r2,0.042355,0.057158,0.041478


In [16]:
sum_metrics = pd.DataFrame({'ref':ref_sum_test_metrics,
                'alt': alt_sum_test_metrics, 
                'altref': altref_sum_test_metrics})

sum_metrics.to_csv('../res/metrics/test_deepset_norho/sum_metrics.csv')
sum_metrics

,ref,alt,altref
loss,22509.743899,100.776833,11.577704
spearman_corr,-0.040267,0.078616,0.221082
pearson_corr,-0.007305,0.042932,0.185423
mae,130.014893,6.597245,2.832499
rmse,150.032486,10.038767,3.402603
r2,-12829.163086,-60.949894,-6.117087
